In [11]:
import requests
import os
import bs4 as bs
#Package pour mettre les données en CSV puis JSON et puis les inserer sur MongoDB
import csv, json
import pandas as pd
import pymongo
#Package Pouvoir se connecter et recuperer les commentaires et les images 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep


#Information de connexion
payload = {
        'username' : 'Elhadjtest Kaisens',
        'password' : 'teste2020'
    }

authentification = " http://www.facebook.com/login?"
#Mettre les sorties dans une liste
text = [] #recupere le texte du post

#Se connecter avec la methode Session de requests pour recuperer le texte du post
with requests.Session() as session:
        post = session.post(authentification, data=payload)
        page = 'https://m.facebook.com/Rirechansons/posts/10156911280547481'
        source = requests.get(page).text
        soup = bs.BeautifulSoup(source, "html.parser")
        text_post = soup.p.text #le texte se trouve dans une balise p
        text.append(text_post) #ajouter la sortie dans notre liste dédiée
        print(text)
       

["L'ancien Président de la République Jacques Chirac est mort à l'âge de 86 ans, vient d'annoncer sa famille😢"]


In [12]:
#Gerer la partie commentaire et image avec du python orienté objet

class FaceBookBot :
    
    def __init__(self):
        options = webdriver.ChromeOptions() #lancer le driver de chrome
        options.add_argument('--disable-notifications') #desactiver les popups
        self.driver = webdriver.Chrome(options=options)
    
    #Entrer sur le site de facebook pour pouvoir saisir ses identifiants de connexion
    def login(self,username, password) :
        self.driver.get (" http://www.facebook.com/login?") 
        sleep(2)
        email_in = self.driver.find_element_by_xpath('//*[@id="email"]')
        email_in.send_Keys(username)
        
        password_in = self.driver.find_element_by_xpath('//*[@id="password"]')
        email_in.send_Keys(password)
        
        login_btn = self.driver.find_element_by_xpath('//*[@id="loginbutton"]')
        login_btn.click()
        
        sleep(2)


### Partie Driver

In [24]:
bot = FaceBookBot() # lancer le driver de chrome
#saisir ses identifiants de connexion
#bot.login(username,password)
bot.driver.get("https://m.facebook.com/story/graphql_permalink/?graphql_id=UzpfSTYzMjQwOTcyNDgwOjEwMTU2OTExMjgwNTQ3NDgx")
page = bot.driver.page_source

In [14]:
#Partie pour les commentaires et les images : le blockage de mon adresse IP par facebook est devenu un frein 
#meme si j'ai vu par la suite que je pouvais passer par facebook developper creer une app et recuperer le token 

#def pageComments(page):
#    resp = requests.get(page)
#    pageComments = []
    #if resp:
       # tables = soup.findAll('div', {'class': 'ed ce cf'})
       # print(tables.text())
        #for table in tables:
            #comment = table.find('div', {'class': 'eg'}
            #print(comment)
            #pageComments.append(comment)
    #return pageComments

In [20]:
#Recuperation et Affichage des Dataframe
test_df = pd.DataFrame({
       'comment' : text
   })
#print (len(head))
#print (test_df)
test_df.to_csv("facebook_comment.csv", index= False, header = None)
test_df.rename(columns=test_df.iloc[0]).drop(test_df.index[0])

,"L'ancien Président de la République Jacques Chirac est mort à l'âge de 86 ans, vient d'annoncer sa famille😢"


In [21]:
#Conversion de CSV en JSON
df = pd.read_csv('facebook_comment.csv', encoding = "utf-8")   # loading csv file
df.to_json('facebook_comment.json')                               # saving to json file
fbc = open('facebook_comment.json').read()                        # loading the json file 
data = json.loads(fbc)                                    # reading json file 

In [22]:
data

{"L'ancien Président de la République Jacques Chirac est mort à l'âge de 86 ans, vient d'annoncer sa famille😢": {}}

In [23]:
##Mettre les donnees dans Mongo
mng_client = pymongo.MongoClient('localhost', 27017)
mng_db = mng_client['facebook'] # nom de la base mongo
collection_name = 'collections' # mongo db collection name
db_cm = mng_db[collection_name]
with open('facebook_comment.json') as f:
    file_data = json.load(f)
    #a = db_cm.insert_one(data) #cette sortie permet d'inserer le texte du fichier json dans la base mongodb 
    #client.close()

In [26]:
# Autre methode pour recuperer les elements d'un post
bot = FaceBookBot() # lancer le driver de chrome
bot.post_ID = "10156911280547481"
#print(bot.post_content())
#print(bot.post_shares())
#print(bot.post_likess())